In [ ]:
import numpy as np
import os
import plotly.graph_objects as go
from tqdm import tqdm
import pandas as pd

import chimera_fgo.util.general as general
from chimera_fgo.util.io import read_lidar_bin, read_gt
from chimera_fgo.util.kitti import process_kitti_gt, load_icp_results, load_sv_positions
from chimera_fgo.util.geometry import euler_to_R

import symforce
try:
    symforce.set_epsilon_to_symbol()
except symforce.AlreadyUsedEpsilon:
    print("Already set symforce epsilon")
    pass 

import symforce.symbolic as sf
from chimera_fgo.chimera_fgo import chimera_fgo

%load_ext autoreload
%autoreload 2

np.set_printoptions(suppress=True, precision=3)

In [ ]:
# ================================================= #
#                    PARAMETERS                     #
# ================================================= #
params = {}
params['MITIGATION'] = True
params['ATTITUDE'] = False

params['N_SHIFT'] = 10
params['N_WINDOW'] = 100
params['GPS_RATE'] = 10

params['TRAJLEN'] = 2000   

params['ATTACK_START'] = params['TRAJLEN'] // 2
params['ALPHA'] = 0.001  # False alarm (FA) rate

params['PR_SIGMA'] = 6.0  # [m]
params['PR_SPOOFED_SIGMA'] = 1e5  # [m]

# Lidar odometry covariance
ODOM_R_SIGMA = 0.01  # so(3) tangent space units
ODOM_T_SIGMA = 0.05  # [m]
ODOM_SIGMA = np.ones(6)
ODOM_SIGMA[:3] *= ODOM_R_SIGMA
ODOM_SIGMA[3:] *= ODOM_T_SIGMA
params['ODOM_SIGMA'] = ODOM_SIGMA

# Attitude measurement standard deviations
ATT_SIGMA_SCALE = 0.2
ATT_SIGMA = ATT_SIGMA_SCALE * np.ones(3)  # so(3) tangent space units (~= rad for small values)
params['ATT_SIGMA'] = ATT_SIGMA

params['kitti_seq'] = '0018'
params['MAX_BIAS'] = 100 

In [ ]:
output = chimera_fgo(params)

In [ ]:
fgo_positions = output['fgo_positions']

# Plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=fgo_positions[:, 0], y=fgo_positions[:, 1], mode='lines', name='FGO'))
fig.show()

In [ ]:
import tracemalloc

def foo():
    l = []
    l.append(np.arange(10000))
    

def bar():
    foo()

for i in range(100):
    tracemalloc.start()
    bar()
    print(tracemalloc.get_traced_memory())